In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy
from neuralnilm.data.datathread import DataThread

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename='/data/mine/vadeec/merged/ukdale.h5',
        buildings=[1, 2],
        sample_period=6,
        window=("2013-04-01", "2013-05-01")
    )
}

nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

Loading kettle for UK-DALE_building_1...


INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...


Loaded 164 kettle activations from UK-DALE_building_1.


INFO:neuralnilm.data.loadactivations:Loaded 164 kettle activations from UK-DALE_building_1.


Loading microwave for UK-DALE_building_1...


...

INFO:neuralnilm.data.loadactivations:Loaded 0 microwave activations from UK-DALE_building_2.


Loading washing machine for UK-DALE_building_2...


INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_2...


Loaded 0 washing machine activations from UK-DALE_building_2.


INFO:neuralnilm.data.loadactivations:Loaded 0 washing machine activations from UK-DALE_building_2.


In [3]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance='kettle',
    seq_length=2048
)

In [4]:
seq = source.get_sequence(enable_all_appliances=True)

In [5]:
all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

True

In [6]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#plt.show()

In [7]:
sample = source.get_batch(num_seq_per_batch=1024)

In [8]:
pipeline = DataPipeline(
    source, 
    num_seq_per_batch=64,
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

In [9]:
report = pipeline.report()
report['activations'] = LOADER_CONFIG
report

{'DataPipeline': {'input_processing': [],
  'num_seq_per_batch': 64,
  'source': {'SyntheticAggregateSource': {'allow_incomplete_distractors': True,
    'allow_incomplete_target': True,
    'distractor_inclusion_prob': 1.0,
    'include_incomplete_target_in_output': True,
    'rng_seed': None,
    'seq_length': 2048,
    'target_appliance': 'kettle',
    'target_inclusion_prob': 1.0,
    'uniform_prob_of_selecting_each_model': True}},
  'target_processing': [{'DivideBy': {'divisor': 291.4845}}]},
 'activations': {'nilmtk_activations': {'appliances': ['kettle',
    'microwave',
    'washing machine'],
   'buildings': [1, 2],
   'filename': '/data/mine/vadeec/merged/ukdale.h5',
   'sample_period': 6,
   'window': ('2013-04-01', '2013-05-01')}}}

In [10]:
data_thread = DataThread(pipeline, enable_all_appliances=False)

In [11]:
data_thread.start()

In [12]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {'dense_layer_0': 100}
    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    final_dense_layer = DenseLayer(
        dense_layer_0,
        num_units=target_shape[1] * target_shape[2]
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [13]:
from neuralnilm.net import Net

batch = data_thread.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [14]:
net.get_symbolic_input()

input

In [15]:
net.get_symbolic_output()

Reshape{3}.0

In [16]:
output = net.deterministic_output_func(batch.after_processing.input)

In [18]:
output.shape

(64, 2048, 1)